# League of Legends 

**Name(s)**: (Calvin Truong, Jake Wanderer)

**Website Link**: (your website link)

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

import plotly.express as px
import matplotlib.pyplot as plt
pd.options.plotting.backend = 'plotly'

# from dsc80_utils import * # Feel free to uncomment and use this.

In [ ]:
lol_fp = Path('data') / '2025_lol_data.csv'
matches = pd.read_csv(lol_fp)

In [ ]:
gold_dif_vs_win_percentage = matches.columns[matches.columns.str.contains('golddiffat')].tolist()
#gold_dif_vs_win_percentage.extend(['gameid','result'])
gold_dif_vs_win_percentage.append('result')
gold_differences = matches[sorted(gold_dif_vs_win_percentage)]
#gold_differences.head(12)
gold_columns = [col for col in gold_differences.columns if col.startswith('golddiffat')]
gold_columns

## Step 1: Introduction

Q1: Approximately how much gold at 10 minutes is a first dragon "worth" and void grubs (in terms of winning the game)? Q2: What is more important..Void Grubs or First Dragon? Also herald or second dragon? Q3: What side is more likely to win the game, and how is it related to who gets the objectives (e.g. dragons, baron)? Q4: How well do heavily banned champions actually perform?

We are likely going to put more emphasis on question 1. We can compare the number of drakes and the number of drakes the opponent has. For our visualization, we should try...For the bivariate analysis, we need two plots --> two good ones could be one with a plot of gold difference and win percentage (and basically have a different line/different color for each gold difference at the different minute intervals) and one with number of dragons vs. number of opponent dragons

## Step 2: Data Cleaning and Exploratory Data Analysis

In [ ]:
lol_fp = Path('data') / '2025_lol_data.csv'
matches = pd.read_csv(lol_fp)

In [ ]:
# gold_dif_and_win_percentage = matches.columns[matches.columns.str.contains('golddiffat')].tolist()
# gold_dif_and_win_percentage.append('result')
# gold_differences = matches[sorted(gold_dif_vs_win_percentage)]
# gold_columns = [col for col in gold_differences.columns if col.startswith('golddiffat')]

#BELOW IS THE ONE I WAS USING PREVIOUSLY
cleaned = matches.copy(deep=False)
gold_info = matches.columns[matches.columns.str.contains('gold')]
gold_df = matches[gold_info]
#cleaned = cleaned.set_index(['gameid','gameid', 'league','side']).loc['11715-11715_game_1'][gold_info]
cleaned = cleaned.set_index(['gameid', 'league','side'])[gold_info]
cleaned = cleaned.reset_index()
cleaned

# cols = [
#     "league", "side",
#     "goldat10", "goldat15", "goldat20", "goldat25",
#     "golddiffat10", "golddiffat15", "golddiffat20", "golddiffat25",
#     "firstdragon",
#     "elementaldrakes", "opp_elementaldrakes",
#     "dragons", "opp_dragons",
#     "result"
# ]
# cleaned = matches.copy(deep=False)
# cleaned = cleaned[cols]
# cleaned['']
# cleaned

In [ ]:
gold_dif_vs_win_percentage = matches.columns[matches.columns.str.contains('golddiffat')].tolist()
gold_dif_vs_win_percentage.append('result')
gold_differences = matches[sorted(gold_dif_vs_win_percentage)]
gold_columns = [col for col in gold_differences.columns if col.startswith('golddiffat')]
for col in gold_columns:
    fig = px.histogram(
        gold_differences,
        x=col,
        title=f'Gold Difference Distribution at {col[10:]} Minutes',
        labels={col: f'Gold Differences at {col[10:]} mins.'}
    )
    fig.show()


In [ ]:
# melted = gold_differences.melt(
#     id_vars=['result'],
#     var_name='Time In-Game', 
#     value_name='Gold Difference'
# )
# melted['Result'] = melted_gold['result'].map({0: 'Loss', 1: 'Win'})

# fig2 = px.box(
#     melted, 
#     x='Time In-Game', 
#     y='Gold Difference', 
#     color='Result',
#     title='Gold Difference Distribution - Wins vs Losses'
# )
# fig2.show()

gold_columns = [col for col in gold_differences.columns if col.startswith('golddiffat')]
for col in gold_columns:
    fig = px.scatter(gold_differences, x=col, y='result')
    mean_loss = gold_differences.loc[gold_differences['result'] == 0, col].mean()
    mean_win = gold_differences.loc[gold_differences['result'] == 1, col].mean()
    
    # Add vertical lines
    fig.add_vline(x=mean_loss, line_dash="dash", line_color="red",
                  annotation_text="Mean Loss", annotation_position="top left")
    fig.add_vline(x=mean_win, line_dash="dash", line_color="green",
                  annotation_text="Mean Win", annotation_position="top right")
    fig.show()

#best to do scatter plot, win or lose is going to be a 1 or 0, gold difference on the x axis, result on the y axis

## Step 3: Assessment of Missingness

In [ ]:
cleaned['golddiffat10_missing'] = cleaned['golddiffat10'].isna()
league_gold10_prop = (
    cleaned
    .groupby(['league', 'golddiffat10_missing'])
    .size()
    .reset_index(name='count')
    .groupby('golddiffat10_missing', group_keys=False)
    .apply(lambda g: g.assign(proportion=g['count'] / g['count'].sum()))
)
fig = px.bar(
    league_gold10_prop,
    x='proportion',
    y='league',
    color='golddiffat10_missing',
    barmode='group',
    orientation='h',
    labels={'golddiffat10_missing': 'golddiffat10 is missing'}
)
fig.update_layout(
    title='Proportion of Leagues by Missingness of golddiffat10',
    xaxis_title='Proportion',
    yaxis_title='League',
    height=900,
    width=900
)
fig.show()


In [ ]:
def tvd(p, q):
    return 0.5 * np.abs(p - q).sum()

In [ ]:
obs = league_gold10_prop.pivot(
    index='league',
    columns='golddiffat10_missing',
    values='proportion'
).fillna(0)
observed_stat = tvd(obs[False], obs[True])
stats = []
for i in range(1000):
    shuffled = cleaned.copy()
    shuffled['golddiffat10_missing'] = np.random.permutation(cleaned['golddiffat10_missing'])
    temp = (
        shuffled
        .groupby(['golddiffat10_missing', 'league'])
        .size()
        .reset_index(name='count')
    )
    temp['proportion'] = (
        temp.groupby('golddiffat10_missing')['count']
        .transform(lambda x: x / x.sum())
    )
    temp_pivot = temp.pivot(
        index='league',
        columns='golddiffat10_missing',
        values='proportion'
    ).fillna(0)
    stat = tvd(temp_pivot[False], temp_pivot[True])
    stats.append(stat)
p_value = (np.array(stats) >= observed_stat).mean()
print(p_value, np.mean(stats))

In [ ]:
observed_stat

In [ ]:
stats_array = np.array(stats)
plt.figure(figsize=(10,5))
plt.hist(stats_array, bins=30, color='blue')
plt.axvline(observed_stat, color='red', linestyle='solid', linewidth=2, label='Observed TVD')
plt.xlabel('Total Variation Distance (TVD)')
plt.ylabel('Probability')
plt.title('Empirical Distribution of TVD')
plt.legend()
plt.show()

In [ ]:
cleaned['golddiffat10_missing'] = cleaned['golddiffat10'].isna()
side_gold10_prop = (
    cleaned
    .groupby(['side', 'golddiffat10_missing'])
    .size()
    .reset_index(name='count')
    .groupby('golddiffat10_missing', group_keys=False)
    .apply(lambda g: g.assign(proportion=g['count'] / g['count'].sum()))
)
fig = px.bar(
    side_gold10_prop,
    x='proportion',
    y='side',
    color='golddiffat10_missing',
    barmode='group',
    orientation='h',
    labels={'golddiffat10_missing': 'golddiffat10 is missing'}
)
fig.update_layout(
    title='Proportion of Leagues by Missingness of golddiffat10',
    xaxis_title='Proportion',
    yaxis_title='Side',
    height=900,
    width=900
)
fig.show()


In [ ]:
obs2 = side_gold10_prop.pivot(
    index='side',
    columns='golddiffat10_missing',
    values='proportion'
).fillna(0)
observed_stat2 = tvd(obs2[False], obs2[True])
stats2 = []
for i in range(1000):
    shuffled = cleaned.copy()
    shuffled['golddiffat10_missing'] = np.random.permutation(cleaned['golddiffat10_missing'])
    temp = (
        shuffled
        .groupby(['golddiffat10_missing', 'side'])
        .size()
        .reset_index(name='count')
    )
    temp['proportion'] = (
        temp.groupby('golddiffat10_missing')['count']
        .transform(lambda x: x / x.sum())
    )
    temp_pivot = temp.pivot(
        index='side',
        columns='golddiffat10_missing',
        values='proportion'
    ).fillna(0)
    stat2 = tvd(temp_pivot[False], temp_pivot[True])
    stats2.append(stat2)
p_value2 = (np.array(stats2) >= observed_stat2).mean()
print(p_value2, np.mean(stats2), observed_stat2)

In [ ]:
stats_array2 = np.array(stats2)
plt.figure(figsize=(10,5))
plt.hist(stats_array2, bins=30, color='blue')
plt.axvline(observed_stat, color='red', linestyle='solid', linewidth=2, label='Observed TVD')
plt.xlabel('Total Variation Distance (TVD)')
plt.ylabel('Probability')
plt.title('Empirical Distribution of TVD')
plt.legend()
plt.show()

## Step 4: Hypothesis Testing

In [ ]:
1+1

## Step 5: Framing a Prediction Problem

In [ ]:
# TODO

## Step 6: Baseline Model

In [ ]:
# TODO

## Step 7: Final Model

In [ ]:
# TODO

## Step 8: Fairness Analysis

In [ ]:
# TODO